<div style="text-align:center"><span style="font-size:2em; font-weight: bold;"> Lecture 7—Training</span></div>

# Programming: Quadratic Programming

When performing optimization, it is sometimes possible to reduce a given problem to a linear or quadratic programming problem. Such problems have extremely fast solutions, which we can use to speed up our code. The downside is that we have to do a little math to put them in this form.

## Linear Programming

$$
\begin{align}
\min_{x,s}\;&c'x \\
\text{subject to}\;& Gx+s=h \\
& Ax=b \\
& s\geq0
\end{align}
$$

## Quadratic Programming

$$
\begin{align}
\min_{x}\;&(1/2)x'Px+q'x \\
\text{subject to}\;& Gx\leq h \\
& Ax=b
\end{align}
$$

Least squares regression can be expressed as a QP problem:
$$
\min_\beta e'e = \min_\beta (y-X\beta)'(y-X\beta) = \min_\beta y'y-2y'X\beta+\beta'X'X\beta
$$
$$
\min_\beta \beta'X'X\beta-2y'X\beta
$$
$$
\min_\beta (1/2)\beta'X'X\beta-y'X\beta
$$
$$
P = X'X, q=-X'y
$$

# Data Science

## $L_1$ Regularization

Given a model:
$$y=\mathbf X\beta+e$$
The $L_1$ regularization of the model is defined by:
$$
\begin{align}
\min_\beta \;& e'e \\
\text{subject to}\;&\Vert\beta\Vert_1 \leq T
\end{align}
$$
Which is equivalent to the summation notation version:
$$
\begin{align}
\min_\beta \;& \sum_{i=1}^n e_i^2\\
\text{subject to}\;&\sum_{j=1}^r\vert\beta_j\vert \leq T
\end{align}
$$
Solving for $\beta$ we obtain:
$$
\begin{align}
\min_\beta \;& (1/2)\beta'\mathbf X'\mathbf X \beta-y'\mathbf X \beta\\
\text{subject to}\;&\mathbf 1_r'\vert\beta\vert \leq T
\end{align}
$$ 
To put this in terms where we can use the quadratic programming format we need to split the betas into their positive and negative components: $\beta=\beta_+-\beta_-$ So we define $\beta_\pm = [\beta_+',\beta_-']'$
$$
\begin{align}
\min_{\beta_{\pm}} \;& (1/2)\beta_\pm'\left(
\begin{bmatrix}
1 & -1 \\
-1 & 1 
\end{bmatrix}
\otimes\mathbf X'\mathbf X \right)\beta_\pm-\left(
\begin{bmatrix}
1 & -1
\end{bmatrix}
\otimes y'\mathbf X \right) \beta_{\pm}\\
\text{subject to}\;&\mathbf 1_{2r}'\beta_\pm \leq T\\
& -\mathbf I_{2r} \beta_{\pm}\leq 0
\end{align}
$$

In [18]:
!pip install --user cvxopt

DEPRECATION: arcgis 1.9.1 has a non-standard dependency specifier keyring<=21.8.*,>=19. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of arcgis or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: celery 5.1.0 has a non-standard dependency specifier pytz>dev. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of celery or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version num

In [19]:
import numpy as np
import pandas as pd
import cvxopt as cvx
import cvxopt.solvers as solv
from scipy.stats import zscore

df = pd.read_csv('BWGHT.csv')
npx = df[['cigs','faminc','male','white']].values
npy = df['bwght'].values
ones = np.ones((npx.shape[0],1))
npx = np.hstack((ones,npx))

ModuleNotFoundError: No module named 'cvxopt'

In [16]:
thresh = 100
def solve_lasso(x,y,thresh):
    n,r = x.shape
    P = np.kron(np.array([[1,-1],[-1,1]]),x.T@x)
    q = -np.kron(np.array([[1],[-1]]),x.T@y.reshape(-1,1))
    G_1 = -np.eye(2*r)
    h_1 = np.zeros((2*r,1))
    G_2 = np.ones((1,2*r))
    h_2 = np.array([[thresh]])
    G = np.vstack((G_1,G_2))
    h = np.vstack((h_1,h_2))
    opt = solv.qp(cvx.matrix(P),cvx.matrix(q),cvx.matrix(G),cvx.matrix(h))
    opt = np.array(opt['x'])
    return opt[:r,0]-opt[r:,0]
np.abs(solve_lasso(npx,npy,thresh)).sum()

NameError: name 'npx' is not defined

In [ ]:
from cleands import *
least_squares_regressor(npx,npy).params

In [ ]:
np.abs(least_squares_regressor(npx,npy).params).sum()

In [ ]:
solve_lasso(npx,npy,thresh)

In [ ]:
thresh = 130
np.abs(solve_lasso(npx,npy,thresh)).sum()

In [ ]:
least_squares_regressor(npx,npy).params,solve_lasso(npx,npy,thresh)

In [ ]:
from cleands import *

class l1_regularization_regressor(least_squares_regressor):
    def __init__(self,x,y,thresh:float,*args,**kwargs):
        super(l1_regularization_regressor,self).__init__(x,y,thresh=thresh,*args,**kwargs)
        self.threshold=thresh
    def __fit__(self,x,y,thresh:float,*args,**kwargs):
        if x[:,0].var()==0:
            dx = x[:,1:]-x[:,1:].mean(0)
            dy = y-y.mean(0)
            outp = solve_lasso(dx,dy,thresh)
            intc = y.mean(0)-x[:,1:].mean(0)@outp.reshape(-1,1)
            return np.concatenate([intc,outp])
        else:
            return solve_lasso(x,y,thresh)
        
params = l1_regularization_regressor(npx,npy,thresh=5).params
print(params)
print(np.abs(params[1:]).sum())

In [ ]:
least_squares_regressor(npx,npy).params

In [ ]:
np.abs(least_squares_regressor(npx,npy).params[1:]).sum()

## Cross-validation

In [ ]:
def mean_squared_error(model,x,y):
    return ((y-model.predict(x))**2).mean()
def k_fold_cross_validation(model,x,y,folds:int=5,seed=None,statistic=mean_squared_error):
    n,r = x.shape
    deck = np.arange(n)
    outp = []
    if seed is not None: np.random.seed(seed)
    np.random.shuffle(deck)
    for i in range(folds):
        test = deck[int(i*n/folds):int((i+1)*n/folds)]
        train_lower = deck[:int(i*n/folds)]
        train_upper = deck[int((i+1)*n/folds):]
        train = np.concatenate((train_lower,train_upper))
        modl = model(x[train],y[train])
        mspe = statistic(modl,x[test],y[test])
        outp += [mspe]
    return np.array(outp)
k_fold_cross_validation(least_squares_regressor,npx,npy,folds=10,seed=90210).mean()

In [ ]:
np.sqrt(k_fold_cross_validation(least_squares_regressor,npx,npy,folds=10,seed=90210).mean())

In [ ]:
npy.std()

In [ ]:
(20.346630897082896-19.926533302221987)/19.926533302221987

In [ ]:
model = lambda x,y: l1_regularization_regressor(x,y,thresh=8)
np.sqrt(k_fold_cross_validation(model,npx,npy,folds=10,seed=90210).mean())

In [ ]:
class l1_cross_validation_regressor(l1_regularization_regressor):
    def __init__(self,x,y,max_thresh=None,folds:int=5,statistic=mean_squared_error,seed=None,*args,**kwargs):
        default_state = solv.options.get('show_progress',True)
        solv.options['show_progress'] = False
        if max_thresh==None: max_thresh = np.abs(least_squares_regressor(x,y).params[1:]).sum()
        outp = []
        for lam in np.linspace(0,1,100):
            model = lambda x,y: l1_regularization_regressor(x,y,thresh=lam*max_thresh)
            mse = k_fold_cross_validation(model,x,y,folds=folds,statistic=statistic,seed=seed).mean()
            outp += [(mse,lam)]
        outp = np.array(outp)
        lam = outp[outp[:,0].argmin(),1]
        thresh = lam*max_thresh
        solv.options['show_progress'] = default_state
        super(l1_cross_validation_regressor,self).__init__(x,y,thresh=thresh,*args,**kwargs)
        self.statistic = outp[outp[:,0].argmin(),0]
        self.max_threshold = max_thresh
        self.lambda_value = lam
model = l1_cross_validation_regressor(npx,npy,folds=10,seed=90210)
print(model.params)
print(model.threshold,model.lambda_value,np.sqrt(model.statistic))

In [ ]:
(19.926533302221987-19.918374979982396)/19.918374979982396

In [ ]:
least_squares_regressor(npx,npy).params

# Programming challenges

##  Tree Simulation

Write a monte carlo simulation comparing the accuracy of tree based models to that of regression for categorical data.

## Recursive partitioning until non-rejection

Modify our recursive partitioning code to test the null that the two groups are equal and stop splitting when the null cannot be rejected. Bonus points if you use the Bonferonni correction when making the decision to split.

In [ ]:
class rpart(prediction_model):
    def __init__(self,x,y,sign_level=0.95,max_level=None,level=''):
        super(rpart,self).__init__(x,y)
        self.max_level = max_level
        self.level = level
        if max_level!=None and len(level)+1==max_level:
            self.__make_terminal_tree__()
            return
        xvars = np.arange(self.n_feat)
        outp = []
        for i in xvars:
            outp += [self.__calc_RSS_and_split__(x[:,i])]
        outp = np.array(outp)
        var = outp[:,0].argmin()
        self.RSS = outp[var,0]
        self.split_var = var
        self.split_value = outp[var,1]
        if max_level==None:
            xvar = (x[:,var]>self.split_value).astype(int)
            xvar = np.hstack((np.ones((self.n_obs,1)),xvar.reshape(-1,1)))
            try:
                model = least_squares_regressor(xvar,y)
            except:
                self.__make_terminal_tree__()                
                return
            tstat = model.params/np.sqrt(np.diag(model.vcov_params))
            tstat = -np.abs(tstat[1])
            critv = sps.t.ppf((1-sign_level)/2/2**(len(level)+1),df=self.n_obs-2)
            if tstat>=critv:
                self.__make_terminal_tree__()
                return
        self.left = rpart(x[x[:,var]<=self.split_value,:],y[x[:,var]<=self.split_value],max_level=max_level,level=level+'L')
        self.right = rpart(x[x[:,var]>self.split_value,:],y[x[:,var]>self.split_value],max_level=max_level,level=level+'R')
    def __make_terminal_tree__(self):
        self.RSS = np.sum((self.y-self.y.mean())**2)
        self.split_var = None
        self.split_value = None
        self.left = None
        self.right = None
    def __calc_RSS_and_split__(self,var):
        vmin = var.min()
        vmax = var.max()
        width = (vmax-vmin)/50
        outp = []
        for split in np.linspace(vmin+width,vmax-width,48):
            left = self.y[var<=split]
            right = self.y[var>split]
            rssleft = ((left-left.mean())**2).sum() if left.shape[0]>0 else 0
            rssright = ((right-right.mean())**2).sum() if right.shape[0]>0 else 0
            outp += [(rssleft+rssright,split)]
        outp = np.array(outp)
        return outp[outp[:,0].argmin(),:]
    def __str__(self):
        if self.left==None and self.right==None:
            outp = '{0} RSS: {1}; Prediction: {2}\n'.format(self.level,self.RSS,self.y.mean())
        else:
            outp = '{0} Variable: {1}; Split: {2}; RSS: {3}\n'.format(self.level,self.split_var,self.split_value,self.RSS)
            outp += str(self.left)
            outp += str(self.right)
        return outp
    def predict(self,newx):
        n = newx.shape[0]
        if self.left==None and self.right==None:
            return np.full(shape=(n,),fill_value=self.y.mean())
        outp = np.zeros((n,))
        outp[newx[:,self.split_var]<=self.split_value] = self.left.predict(newx[newx[:,self.split_var]<=self.split_value,:])
        outp[newx[:,self.split_var]>self.split_value] = self.right.predict(newx[newx[:,self.split_var]>self.split_value,:])
        return outp

# Homework 3 Solutions

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from cleands import *
from itertools import product

nsim = 1000
sigvec = [0.5,1,2]
alphavec = [-0.5,0,0.5]
rhovec = np.linspace(-0.75,0.75,7).tolist()
nvec = np.linspace(10,100,10,dtype=int).tolist()
bigvec = [sigvec,alphavec,rhovec,nvec]
outp = np.zeros([nsim,*[len(item) for item in bigvec]])
outps = np.zeros([nsim,*[len(item) for item in bigvec]])
outpa = np.zeros([nsim,*[len(item) for item in bigvec]])
outpr = np.zeros([nsim,*[len(item) for item in bigvec]])
outpn = np.zeros([nsim,*[len(item) for item in bigvec]])

In [ ]:
for (si,sig),(ai,alpha),(ri,rho),(ni,n) in product(*[enumerate(item) for item in bigvec]):
    print(sig,alpha,rho,n)
    for isim in range(nsim):
        x = np.random.normal(size=(n+1+100,))*sig
        x[0] += alpha
        for i in range(1,n+1+100):
            x[i] = alpha+rho*x[i-1]+x[i]
        x = x[100:]
        xmat = [np.ones((n,1)),x[:-1].reshape(-1,1)]
        xmat = np.hstack(xmat)
        ymat = x[1:]
        b = np.linalg.solve(xmat.T@xmat,xmat.T@ymat)
        outp[isim,si,ai,ri,ni] = b[1]
        outps[isim,si,ai,ri,ni] = sig
        outpa[isim,si,ai,ri,ni] = alpha
        outpr[isim,si,ai,ri,ni] = rho
        outpn[isim,si,ai,ri,ni] = n

In [ ]:
bias = outp.mean(0)-outpr.mean(0)
rho = outpr.mean(0)
sig = outps.mean(0)
alpha = outpa.mean(0)
n = outpn.mean(0)

In [ ]:
df = pd.DataFrame({'bias':bias.reshape(-1),'rho':rho.reshape(-1),'sig':sig.reshape(-1),'alpha':alpha.reshape(-1),'n':n.reshape(-1)})

In [ ]:
df

In [ ]:
plt.scatter(df['rho'],df['bias'],c=df['n']);

In [ ]:
plt.scatter(df['alpha'],df['bias'],c=df['n']);

In [ ]:
plt.scatter(df['sig'],df['bias'],c=df['n']);

In [ ]:
plt.scatter(df['n'],df['bias'],c=df['rho']);

In [ ]:
df['1/n'] = 1/df['n']
df['rho/n'] = df['rho']/df['n']

In [ ]:
model = LeastSquaresRegressor(*add_intercept(['rho','n'],'bias',df))
model.glance

In [ ]:
model = LeastSquaresRegressor(*add_intercept(['rho','1/n'],'bias',df))
model.glance

In [ ]:
model = LeastSquaresRegressor(*add_intercept(['rho','1/n','rho/n'],'bias',df))
model.glance

In [ ]:
model.tidy

$$\text{bias}\left[\hat\rho\right] = 0.00117+-0.00773\rho+-1.112\frac{1}{n}+-2.538\frac{\rho}{n}$$
$$\text{bias}\left[\hat\rho\right] = \frac{-1.112}{n}+\frac{-2.538\rho}{n}$$
$$\text{bias}\left[\hat\rho\right] = -\frac{1.112+2.538\rho}{n}$$

In [5]:
r = 2
-np.eye(2*r)

array([[-1., -0., -0., -0.],
       [-0., -1., -0., -0.],
       [-0., -0., -1., -0.],
       [-0., -0., -0., -1.]])

In [6]:
np.ones((1,2*r))

array([[1., 1., 1., 1.]])

In [8]:
G_1 = -np.eye(2*r)
h_1 = np.zeros((2*r,1))
G_2 = np.ones((1,2*r))
h_2 = np.array([[10]])
G = np.vstack((G_1,G_2))
h = np.vstack((h_1,h_2))

In [9]:
G_1

array([[-1., -0., -0., -0.],
       [-0., -1., -0., -0.],
       [-0., -0., -1., -0.],
       [-0., -0., -0., -1.]])

In [10]:
G_2

array([[1., 1., 1., 1.]])

In [11]:
G

array([[-1., -0., -0., -0.],
       [-0., -1., -0., -0.],
       [-0., -0., -1., -0.],
       [-0., -0., -0., -1.],
       [ 1.,  1.,  1.,  1.]])

In [12]:
h

array([[ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [10.]])